<h1>Project Euler Problem #50: Consecutive Prime Sum</h1>

Project Euler is a website that offers a collection of challenging mathematical and computational problems intended to be solved with computer programs. The problems range in difficulty from relatively simple to highly complex, and they often require creative problem-solving skills and efficient algorithms to solve efficiently. I love it!

Check it out here: https://projecteuler.net

Some problems can be solved by brute-forcing (writing an algorithm to work iteratively and exhaustively until a solution is found), but for this one it didn't make sense to me. Here I break down the problem as I tackled it.

<h2>The problem</h2>

The prime 41, can be written as the sum of six consecutive primes:

$41 = 2 + 3 + 5 + 7 + 11 + 13$

This is the longest sum of consecutive primes that adds to a prime below one-hundred.

The longest sum of consecutive primes below one-thousand that adds to a prime, contains 
21 terms, and is equal to 953.

<b>Which prime, below one-million, can be written as the sum of the most consecutive primes?</b>

The problem can be found here: https://projecteuler.net/problem=50

<h2>A solution!</h2>

First we need to generate a list of primes so that we can generate some sums of them. A commonly used method is to use the Sieve of Eratosthenes.

For more information see this Wikipedia page: https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes

In [1]:
def sieveOfEratosthenes(limit):
    primes = []
    sieve = [True] * (limit + 1)
    sieve[0] = sieve[1] = False
    
    for num in range(2, int(limit ** 0.5) + 1):
        if sieve[num]:
            primes.append(num)
            for multiple in range(num * num, limit + 1, num):
                sieve[multiple] = False
                
    for num in range(int(limit ** 0.5) + 1, limit + 1):
        if sieve[num]:
            primes.append(num)
            
    return primes

And now we can use that function and view the first 20:

In [2]:
# generate list of all prime numbers below 1-million
primeSeq = sieveOfEratosthenes(int(1e6))

# print the length of the list
print(len(primeSeq))

# print the first 20 of those prime numbers
print(primeSeq[0:20])

# print the highest prime below 1-million
print(primeSeq[len(primeSeq) - 1])

78498
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]
999983


Our list of primes is quite long, there are $78498$ of them! The largest prime below 1-million is $999983$. If we're doing a sum of as many consecutive primes as possible with a sum below 1-million that is unlikely to be one of them ;-)

My first move was just to see how many primes could be summed from the first element ($2$).

Here's my chunk of code. It simply sums increasing numbers of primes from element $0$ to element n of primeSeq, and if the sum is prime it records that as a success. It searches until the sum exceeds 1-million. The highest sum that was prime is recorded.

It wasn't necessary to sum every possible primeSeq$[0:1, 2, 3 ... n]$, because a prime is an odd number. We are starting with the first -and only even- prime, $2$. Any other two primes when added together will give and even number, and added to $2$ that will also be even and <b>cannot be prime</b>:

$2 + 3 = 5$ 

$2 + 3 + 5 = 8$

$2 + 3 + 5 + 7 = 15$

$2 + 3 + 5 + 7 + 11 = 26$

So, from the starting prime of $2$, we only need sum that with an odd number of other primes, therefore halving the number of iterations the algorithm would need to perform. In our loop then, we will be looking only at an even number of primes ($2$ + and odd number of others).

So here's my code and let's find out the sum of the largest number of consecutive primes, starting from $2$, that we can get below 1-million:

In [3]:
### if first prime == 2 (even number), then starting iteration is 2 and 
### increases by 2 at each iteration

numberOfPrimes = 0
highestPrime = 0
consituentPrimes = []
maxLimit = 1000000
currentIteration = 2
while currentIteration < maxLimit: # something more like repeat?
    currentIteration += 2
    evalNum = sum(primeSeq[0:currentIteration])

    if evalNum > maxLimit:
        print("maxLimit exceeded! Stopping...")
        break
    if evalNum in primeSeq:
        primesList = primeSeq[0:currentIteration]
        if len(primesList) > numberOfPrimes:
            highestPrime = evalNum
            numberOfPrimes = len(primesList)
            constituentPrimes = primesList
        
print("Number of primes:")        
print(numberOfPrimes)
print("Maximium prime under 1e6:")
print(highestPrime)

maxLimit exceeded! Stopping...
Number of primes:
536
Maximium prime under 1e6:
958577


Wow, okay, so we hit $958577$ as a consecutive sum of primes from $2$ to $3877$ (the $536^{th}$ prime).

A sum of $536$ primes is the <i>fewest</i> we're looking at. But perhaps a longer sequence and a higher total is possible if we start from $3$ or $5$ or one of the higher primes.

Importantly, there is not that much room for a higher prime than $958577$ as we're not far off 1-million. Being that the upper prime is $3877$ and any larger sequence would involve larger primes we can conservatively figure out how many other start points (from prime 2 ($3$), prime 3 ($5$), etc.) we need try.

Let's work that out by taking the difference of 1-million minus the highest prime found, and dividing that by the highest prime in our sequence (which will be lower than any other subsequent prime):

In [4]:
(1000000 - 958577) / 3877

10.684291978333762

Okay, so we can only beat the maximum number of primes if our starting prime is less than a conservative guesstimate of prime $10$ or $11$ as our sum sill exceed 1-million. We can be sure that the total number of primes is between $536$ and $(536 + 10 =)$ 546.

As we are checking for a continuous sequence starting from other primes, they are all now odd numbers and so we need only to check a sum of an odd number of primes (as an even number of primes will produce a sum divisible by $2$ which cannot be prime). Again, this saves a bit of computation.

We will now start iteratively removing the first prime from primeSeq ($3, 5, 7, ...$) and seeing if we can beat our total number of primes.

The code from above has been slightly modified so that current iteration is $1$ (odd) and not $2$ even. And adding $2$ to the currentIteration in the loop will always produce an odd number.

In [5]:
primeSeqOdd = primeSeq[1:len(primeSeq)]
for i in range(1, 10):
    primeSeqOdd = primeSeqOdd[1:len(primeSeqOdd)]

    currentIteration = 1
    while currentIteration < maxLimit: # something more like repeat?
        currentIteration += 2
        evalNum = sum(primeSeqOdd[0:currentIteration])
    
        if evalNum > maxLimit:
            print("maxLimit exceeded! Stopping...")
            break
        if evalNum in primeSeqOdd:
            primesList = primeSeqOdd[0:currentIteration]
            if len(primesList) > numberOfPrimes:
                highestPrime = evalNum
                numberOfPrimes = len(primesList)
                constituentPrimes = primesList
        
print("Number of primes:")        
print(numberOfPrimes)
print("Maximium prime under 1e6:")
print(highestPrime)

maxLimit exceeded! Stopping...
maxLimit exceeded! Stopping...
maxLimit exceeded! Stopping...
maxLimit exceeded! Stopping...
maxLimit exceeded! Stopping...
maxLimit exceeded! Stopping...
maxLimit exceeded! Stopping...
maxLimit exceeded! Stopping...
maxLimit exceeded! Stopping...
Number of primes:
543
Maximium prime under 1e6:
997651


And there we go! The highest number of consecutive primes that can be summed to a prime number less than 1-million...